# Riconoscimento ottico dei caratteri

![Un robot che legge un giornale](./images/ocr.jpg)

Una sfida comune della visione artificiale è quella di rilevare e interpretare il testo in un'immagine. Questo tipo di elaborazione è spesso indicato come *riconoscimento ottico dei caratteri* (OCR).

## Usa il servizio Visione artificiale per leggere il testo in un'immagine

Il servizio cognitivo **Visione artificiale** fornisce supporto per attività OCR, tra cui:

- Un'API **OCR** utilizzabile per leggere il testo in più lingue. Quest'API può essere utilizzata in modo sincrono e funziona bene quando è necessario rilevare e leggere una piccola quantità di testo in un'immagine.
- Un'API **Read** che è ottimizzata per documenti di dimensioni maggiori. Quest'API è usata in modo asincrono e può essere usata sia per il testo stampato che per quello scritto a mano.

Puoi utilizzare questo servizio creando una risorsa di **Visione artificiale** o una risorsa di **Servizi cognitivi**.

Se non l'hai ancora fatto, crea una risorsa di **Servizi Cognitivi** nella tua sottoscrizione di Azure.

> **Nota**: Se disponi già di una risorsa di Servizi Cognitivi, basta aprire la sua pagina **Avvio rapido** nel portale di Azure e copiare la sua chiave e l'endpoint nella cella seguente. Altrimenti, procedi come segue per crearne una.

1. In un'altra scheda del browser, apri il portale Azure all'indirizzo https://portal.azure.com e accedi con il tuo account Microsoft.

2. Fai clic sul pulsante **&#65291;Crea una risorsa**, cerca *Servizi cognitivi* e crea una risorsa di **Servizi cognitivi** con le impostazioni seguenti:
    - **Sottoscrizione**: *La tua sottoscrizione di Azure*.
    - **Gruppo di risorse**: *Seleziona o crea un gruppo di risorse con un nome univoco*.
    - **Area geografica**: *Scegli una qualsiasi area disponibile*:
    - **Nome**: *Immetti un nome univoco*.
    - **Piano tariffario**: S0
    - **Confermo di aver letto e compreso gli avvisi**: Selezionato.
3. Attendi il completamento della distribuzione. Vai quindi alla tua risorsa di servizi cognitivi e, nella pagina **Panoramica**, fai clic sul link per gestire le chiavi per il servizio. Avrai bisogno dell'endpoint e delle chiavi per connetterti alla tua risorsa di servizi cognitivi dalle applicazioni client.

### Ottieni la chiave e l'endpoint per la tua risorsa di Servizi cognitivi

Per usare la risorsa di servizi cognitivi, le applicazioni client hanno bisogno del loro endpoint e della chiave di autenticazione:

1. Nel portale di Azure, nella pagina **Chiavi ed endpoint** per la tua risorsa di servizio cognitivo, copia la **Key1** per la tua risorsa e incollala nel codice sottostante, sostituendo **YOUR_COG_KEY**.
2. Copia l'**endpoint** per la tua risorsa e incollalo nel codice sottostante, sostituendo **YOUR_COG_ENDPOINT**.
3. Esegui il codice nella cella qui sotto facendo clic sul pulsante **Esegui cella** (&#9655;) a sinistra della cella.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

Ora che hai impostato la chiave e l'endpoint, puoi usare la tua risorsa del servizio Visione artificiale per estrarre testo da un'immagine.

Iniziamo con l'API **OCR**, che permette di analizzare sincronicamente un'immagine e leggere qualsiasi testo in essa contenuto. In questo caso, hai un'immagine pubblicitaria della società fittizia di vendita al dettaglio Northwind Traders, in cui è presente del testo. Esegui la cella seguente per leggerlo. 

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
%matplotlib inline

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Read the image file
image_path = os.path.join('data', 'ocr', 'advert.jpg')
image_stream = open(image_path, "rb")

# Use the Computer Vision service to find text in the image
read_results = computervision_client.recognize_printed_text_in_stream(image_stream)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)
plt.axis('off')
plt.imshow(img)

Il testo trovato nell'immagine è organizzato in una struttura gerarchica di aree geografiche, linee e parole, che il codice legge per recuperare i risultati.

Nei risultati, visualizza il testo che è stato letto sopra l'immagine. 

## Visualizza i riquadri delimitatori del testo

I risultati includono anche le coordinate dei *riquadri delimitatori del testo* per le righe di testo e le singole parole trovate nell'immagine. Esegui la cella seguente per vedere i riquadri delimitatori del testo per le linee di testo nell'immagine pubblicitaria che hai recuperato sopra.

In [ ]:
# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Show the position of the line of text
        l,t,w,h = list(map(int, line.bounding_box.split(',')))
        draw.rectangle(((l,t), (l+w, t+h)), outline='magenta', width=5)

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Show the image with the text locations highlighted
plt.axis('off')
plt.imshow(img)

Nel risultato, il riquadro delimitatore del testo per ogni riga di testo è mostrato come un rettangolo sull'immagine.

## Utilizza l'API Read

L'API OCR che hai usato in precedenza funziona bene per immagini con una piccola quantità di testo. Quando hai bisogno di leggere corpi di testo maggiori, come i documenti scansionati, puoi usare l'API **Read**. Per farlo è necessario un processo in più passaggi:

1. Invia un'immagine al servizio Visione artificiale per leggerla e analizzarla in modo asincrono.
2. Attendi il completamento dell'operazione di analisi.
3. Recupera i risultati dell'analisi.

Esegui la cella seguente per utilizzare questo processo e leggere il testo di una lettera scansionata al responsabile di un negozio Northwind Traders.

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'letter.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

Esamina i risultati. È presente una trascrizione completa della lettera, che consiste per lo più in testo stampato con una firma scritta a mano. L'immagine originale della lettera è mostrata sotto i risultati OCR (potrebbe essere necessario scorrere per vederla).

## Leggi il testo scritto a mano

Nell'esempio precedente, la richiesta di analizzare l'immagine specificava una modalità di riconoscimento del testo che ottimizzava l'operazione per il testo *stampato*. Tieni presente che nonostante ciò, la firma scritta a mano è stata letta.

Questa capacità di leggere il testo scritto a mano è estremamente utile. Ad esempio, supponiamo che tu abbia scritto una nota contenente una lista della spesa, e vuoi usare un'applicazione sul tuo telefono per leggere la nota e trascrivere il testo che contiene.

Esegui la cella seguente per vedere un esempio di operazione di lettura di una lista della spesa scritta a mano.

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'note.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## Ulteriori informazioni

Per ulteriori informazioni sull'uso del servizio Visione artificiale per l'OCR, consulta la [documentazione di Visione artificiale](https://docs.microsoft.com/it-it/azure/cognitive-services/computer-vision/concept-recognizing-text)